# A Deep Learning based Efficacy Prediction System for Drug Discovery

In [ ]:
########################################################
# All rights reserved. 
# Author: XIE Zhengwei @ Beijing Gigaceuticals Tech Co., Ltd 
#                      @ Peking University International Cancer Institute
# Contact: xiezhengwei@gmail.com
#
#
########################################################
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from dleps_predictor import DLEPS


Load the SMILES for natural products and FDA approved drugs

In [ ]:

smi = pd.read_csv('../../data/Brief_Targetmol_natural_product_2719')
fda = pd.read_csv('../../data/Brief_FDA-Approved-Drug_961')


In [ ]:
smi




Load the DLEPS model. DLEPS requires up set of genes and down set of genes to calculate cs = a - b , or only one set for calculating a.

In [ ]:
predictor = DLEPS(reverse=False, up_name='../../data/BROWNING_up',down_name='../../data/BROWNING_down',save_exp=None)




The model structure showed followed

In [ ]:
predictor.model[0].summary()



Execute model inference, input requires a SMILES array, output is a cs array with -2 for unsuccessed processing. 
predict function has the followed options:
predict(self, setmean = False, save_onehot=None, load_onehot=None, reverse = True, base = -2, up_name='\', down_name='', save_exp = None)

setmean : False, z score (Change of Transcriptional Profiles, CTP) = expression level - mean, setmean refers to mean of expression files for user inputed chemicals \
save_onehot: None, save onehot array for the current chemical set for future use \
load_onehot: None, load onehot if onehot files have been saved \
reverse: True, do user want to reverse the input up / down gene set, only works for the paired input \
base: -2, default value for error parsed small molecules

In [ ]:
scores = predictor.predict(fda['SMILES'].values)
#scores = predictor.predict(celastrol)

In [ ]:
fda



Visualize the cs scores

In [ ]:
plt.hist(scores,50)
plt.show()

In [ ]:
fda['score'] = scores

In [ ]:
fda

In [ ]:
#fda = fda.set_index('Unnamed: 0')
#fda.to_csv('../../results/fda_HUA_merge.csv')


In [ ]:
smi_scores = predictor.predict(smi['SMILES'].values)

In [ ]:
plt.hist(smi_scores,50)
plt.show()

In [ ]:
smi['score'] = smi_scores

In [ ]:
smi = smi.set_index('Unnamed: 0')
smi